# Import

These requirements are necessary if you launch this notebook from SageMaker instances

In [1]:
"""!pip install mlflow
!pip install pytorch-lightning
!pip install transformers
!pip install tqdm
!pip install sagemaker

!pip install s3fs
!pip install smdebug"""

'!pip install mlflow\n!pip install pytorch-lightning\n!pip install transformers\n!pip install tqdm\n!pip install sagemaker\n\n!pip install s3fs\n!pip install smdebug'

In [2]:
import sys
sys.path.append('../../../')

import os
import sys
import logging
import argparse
from pathlib import Path
from typing import Any, Dict, Optional

In [3]:
from tqdm.auto import tqdm

import torchmetrics
from torchmetrics.functional import accuracy, f1, auroc

import sagemaker
from sagemaker import get_execution_role
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.core.decorators import auto_move_data
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import MLFlowLogger


import matplotlib.pyplot as plt
from pylab import rcParams
from matplotlib import rc

import pandas as pd

Local constants, regarding the data, MLFlow server, paths, etc..: use them

In [4]:
from deep.constants import *
from deep.utils import *

In [5]:
%load_ext autoreload
%autoreload 2

# Data

You can use the data you want. We advise the `pandas` format.

In [6]:
DATA_PATH = os.path.join(
    '..', '..', '..', "data", "frameworks_data", 'data_v0.7.1'
)

"""
original_df = pd.read_csv(os.path.join(DATA_PATH, 'full_dataset.csv'))
augmented_data = pd.read_csv(os.path.join(DATA_PATH, 'generated_text.csv'))"""


tot_df = pd.read_csv(os.path.join(DATA_PATH, 'train_val_df.csv'))
test_df = pd.read_csv(os.path.join(DATA_PATH, 'test_v0.7.1.csv'))
"""full_df = pd.read_csv(os.path.join(DATA_PATH, 'full_dataset_with_translations.csv'))"""

/home/selim/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (7,26,27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


"full_df = pd.read_csv(os.path.join(DATA_PATH, 'full_dataset_with_translations.csv'))"

In [7]:
set(tot_df.entry_id.unique().tolist()).intersection(
set(test_df.entry_id.unique().tolist())
)

{17827, 148224}

In [8]:
"""train_val_ids = pd.concat([train_df, val_df]).entry_id.unique()
train_val_df = full_df[full_df.entry_id.isin(train_val_ids)]

fr_df = train_val_df[['entry_id', 'translation_fr']]\
        .rename(columns={'translation_fr':'excerpt'}).dropna()
en_df = train_val_df[['entry_id', 'translation_en']]\
        .rename(columns={'translation_en':'excerpt'}).dropna()
es_df = train_val_df[['entry_id', 'translation_es']]\
        .rename(columns={'translation_es':'excerpt'}).dropna()

augmented_data = pd.concat([en_df, fr_df, es_df])


print('fr:', fr_df.shape[0], 'en:', en_df.shape[0], 'es:', es_df.shape[0])"""

"train_val_ids = pd.concat([train_df, val_df]).entry_id.unique()\ntrain_val_df = full_df[full_df.entry_id.isin(train_val_ids)]\n\nfr_df = train_val_df[['entry_id', 'translation_fr']]        .rename(columns={'translation_fr':'excerpt'}).dropna()\nen_df = train_val_df[['entry_id', 'translation_en']]        .rename(columns={'translation_en':'excerpt'}).dropna()\nes_df = train_val_df[['entry_id', 'translation_es']]        .rename(columns={'translation_es':'excerpt'}).dropna()\n\naugmented_data = pd.concat([en_df, fr_df, es_df])\n\n\nprint('fr:', fr_df.shape[0], 'en:', en_df.shape[0], 'es:', es_df.shape[0])"

In [9]:
"""def preprocess_columns(tmp_col):
    if str(tmp_col) == 'nan':
        return []
    evaluated_column = literal_eval(tmp_col)
    cleaned_column = [item for item in evaluated_column if str(item)!='NOT_MAPPED' and str(item)!='None']
    return cleaned_column"""

"def preprocess_columns(tmp_col):\n    if str(tmp_col) == 'nan':\n        return []\n    evaluated_column = literal_eval(tmp_col)\n    cleaned_column = [item for item in evaluated_column if str(item)!='NOT_MAPPED' and str(item)!='None']\n    return cleaned_column"

In [10]:
"""from ast import literal_eval
columns = [
     'sectors',
     'severity',
     'age',
     'gender',
     'subpillars_1d', 
     'specific_needs_groups',
     'subpillars_2d', 
    ]
def flatten(t):
    return [item for sublist in t for item in sublist]
for column in columns:
    train_val_df[column] = train_val_df[column].apply(preprocess_columns)
"""

"from ast import literal_eval\ncolumns = [\n     'sectors',\n     'severity',\n     'age',\n     'gender',\n     'subpillars_1d', \n     'specific_needs_groups',\n     'subpillars_2d', \n    ]\ndef flatten(t):\n    return [item for sublist in t for item in sublist]\nfor column in columns:\n    train_val_df[column] = train_val_df[column].apply(preprocess_columns)\n"

In [11]:
"""augmented_data = pd.merge(
    right=train_val_df.drop(columns=[
        'excerpt', 'translation_en', 'translation_fr', 'translation_es']
                           ),
    left=augmented_data[['entry_id', 'excerpt']],
    on='entry_id',
    how='right'
)"""

"augmented_data = pd.merge(\n    right=train_val_df.drop(columns=[\n        'excerpt', 'translation_en', 'translation_fr', 'translation_es']\n                           ),\n    left=augmented_data[['entry_id', 'excerpt']],\n    on='entry_id',\n    how='right'\n)"

In [12]:
"""tot_df = pd.concat([train_val_df, augmented_data])"""

'tot_df = pd.concat([train_val_df, augmented_data])'

In [13]:
"""columns = [
     'sectors',
     'severity',
     'age',
     'gender',
     'subpillars_1d', 
     'specific_needs_groups',
     'subpillars_2d', 
    ]
def flatten(t):
    return [item for sublist in t for item in sublist]
tot_df['column_present'] = tot_df.apply(
    lambda x: [column for column in columns if len(x[column])>2], axis=1
               )"""

"columns = [\n     'sectors',\n     'severity',\n     'age',\n     'gender',\n     'subpillars_1d', \n     'specific_needs_groups',\n     'subpillars_2d', \n    ]\ndef flatten(t):\n    return [item for sublist in t for item in sublist]\ntot_df['column_present'] = tot_df.apply(\n    lambda x: [column for column in columns if len(x[column])>2], axis=1\n               )"

In [14]:
"""tot_df.to_csv(os.path.join(DATA_PATH, 'train_val_df.csv'))"""

"tot_df.to_csv(os.path.join(DATA_PATH, 'train_val_df.csv'))"

In [15]:
"""augmented_data = pd.merge(
    right=original_df.drop(columns=['excerpt']),
    left=augmented_data[['entry_id', 'excerpt']],
    on='entry_id',
    how='right'
)"""

"augmented_data = pd.merge(\n    right=original_df.drop(columns=['excerpt']),\n    left=augmented_data[['entry_id', 'excerpt']],\n    on='entry_id',\n    how='right'\n)"

In [16]:
"""tot_df = pd.concat([original_df, augmented_data])"""

'tot_df = pd.concat([original_df, augmented_data])'

In [17]:
"""from ast import literal_eval"""

'from ast import literal_eval'

In [18]:
"""tot_df['pillars_1d'] = tot_df.subpillars_1d.apply(
    lambda item: [x.split('->')[0] for x in literal_eval(item)]
)
tot_df['pillars_2d'] = tot_df.subpillars_2d.apply(
    lambda item: [x.split('->')[0] for x in literal_eval(item)]
)"""

"tot_df['pillars_1d'] = tot_df.subpillars_1d.apply(\n    lambda item: [x.split('->')[0] for x in literal_eval(item)]\n)\ntot_df['pillars_2d'] = tot_df.subpillars_2d.apply(\n    lambda item: [x.split('->')[0] for x in literal_eval(item)]\n)"

In [19]:
columns = ['excerpt', 'entry_id',
           'column_present',
         'sectors',
         'severity',
         'age',
         'gender',
         'subpillars_1d', 
         'specific_needs_groups',
         'subpillars_2d', 
        ]

tot_df = tot_df[columns]

## Sagemaker Prep

### Session

Configure SageMaker

In [20]:
sess = sagemaker.Session(default_bucket=DEV_BUCKET.name)
role = SAGEMAKER_ROLE
role_arn = SAGEMAKER_ROLE_ARN
tracking_uri = MLFLOW_SERVER

### Bucket upload

You need to upload data to an S3 bucket. 




In [21]:
MLFLOW_SERVER

'http://mlflow-deep-387470f3-1883319727.us-east-1.elb.amazonaws.com/'

In [22]:
sample = True  # To make the computations faster, sample = True.

if sample:
    tot_df = tot_df.sample(n=20_000)
    
job_name = f"pytorch-{formatted_time()}-all-models"  # change it as you prefer
input_path = DEV_BUCKET / 'training' / 'input_data' / job_name  # Do not change this

train_path = str(input_path / 'train.pickle')
val_path = str(input_path / 'val.pickle')


tot_df.to_pickle(train_path, protocol=4)  # protocol 4 is necessary, since SageMaker uses python 3.6
test_df.to_pickle(val_path, protocol=4)

### Estimator Definition

In [23]:
# GPU instances

instances = [
    'ml.p2.xlarge',
    'ml.p3.2xlarge'
]

The hyperparameters are passed as command line arguments to the training script. 

You can add/change them as you like. It's important to keep the `tracking_uri` and the `experiment_name` which are used by MLFlow.

The class `PyTorch` is part of the `SageMaker` python API. The parameters are important and you should probably not change most of them. The ones you may want to change are:

- `instance_type`, specify the instance you want
- `source_dir`, specify your script directory. Try to use global variable as much as possible

In [24]:
from sagemaker.pytorch import PyTorch

instance_type='ml.p3.2xlarge'

hyperparameters={
    'tracking_uri': MLFLOW_SERVER,
    'experiment_name': "pl-trials",
    'max_len': 512,
    'epochs': 3,
    'model_name': 'microsoft/xtremedistil-l6-h256-uncased',
    'tokenizer_name': 'microsoft/xtremedistil-l6-h256-uncased',
    'dropout_rate': 0.4,
    'output_length': 256,
    'training_names':','.join(columns[2:]),
    "instance_type": instance_type,
    'beta_f1': 0.8
}

estimator = PyTorch(
    entry_point='train_mlflow.py',
    source_dir=str('../../../scripts/training/selim/multiclass-lightning'),
    output_path=str(DEV_BUCKET/'models/'),
    code_location=str(input_path),
    instance_type=instance_type,
    instance_count=1,
    role=role,
    framework_version="1.8",
    py_version="py36",
    hyperparameters = hyperparameters,
    job_name=job_name,
#     train_instance_count=2,
#     train_instance_type="ml.c4.xlarge",
)

In [25]:
fit_arguments = {
    'train': str(input_path),
    'test': str(input_path)
}

In [26]:
# Fit the estimator

estimator.fit(fit_arguments, job_name=job_name)

2021-11-05 16:25:19 Starting - Starting the training job...
2021-11-05 16:25:42 Starting - Launching requested ML instancesProfilerReport-1636129516: InProgress
.........
2021-11-05 16:27:24 Starting - Preparing the instances for training............
2021-11-05 16:29:31 Downloading - Downloading input data...
2021-11-05 16:30:05 Training - Downloading the training image....................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-11-05 16:33:29,106 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-11-05 16:33:29,131 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-11-05 16:33:29,147 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-11-05 16:33:29,952 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r requirements.txt


  Created wheel for wrapt: filename=wrapt-1.12.1-cp36-cp36m-linux_x86_64.whl size=69745 sha256=38eced3c3f07e6281e9d349e86077f07e946e4b1e28101e785414c2480f51d6d
  Stored in directory: /root/.cache/pip/wheels/32/42/7f/23cae9ff6ef66798d00dc5d659088e57dbba01566f6c60db63
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-py3-none-any.whl size=3160 sha256=4813f093fef66d99267b1d8052bcf8768a2028216106f8fa9b0c4176dbaf51bf
  Stored in directory: /root/.cache/pip/wheels/6a/f5/9c/f8331a854f7a8739cf0e74c13854e4dd7b1af11b04fe1dde13
Successfully built sagemaker aiobotocore alembic databricks-cli termcolor wrapt idna-ssl
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: numpy
    Fo

[2021-11-05 16:35:07.257 algo-1:79 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-11-05 16:35:07.307 algo-1:79 INFO profiler_config_parser.py:102] User has disabled profiler.
[2021-11-05 16:35:07.308 algo-1:79 INFO json_config.py:91] Creating hook from json_config at /opt/ml/input/config/debughookconfig.json.
[2021-11-05 16:35:07.309 algo-1:79 INFO hook.py:201] tensorboard_dir has not been set for the hook. SMDebug will not be exporting tensorboard summaries.
[2021-11-05 16:35:07.310 algo-1:79 INFO hook.py:255] Saving to /opt/ml/output/tensors
[2021-11-05 16:35:07.310 algo-1:79 INFO state_store.py:77] The checkpoint config file /opt/ml/input/config/checkpointconfig.json does not exist.
[2021-11-05 16:35:07.542 algo-1:79 INFO hook.py:594] name:model.l0.embeddings.word_embeddings.weight count_params:7813632
[2021-11-05 16:35:07.543 algo-1:79 INFO hook.py:594] name:model.l0.embeddings.position_embeddings.weight count_params:131072
[2021-11-05 16:35:07.543 algo-1:79 INFO hook.

#015Validation sanity check: 0it [00:00, ?it/s]#015Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]#015                                                              #015#015Training: 99it [00:00, ?it/s]#015Training:   0%|          | 0/574 [00:00<?, ?it/s]#015Epoch 0:   0%|          | 0/574 [00:00<?, ?it/s] #015Epoch 0:   5%|▌         | 30/574 [00:03<01:05,  8.25it/s]#015Epoch 0:   5%|▌         | 30/574 [00:03<01:05,  8.25it/s, loss=1.260, v_num=0, val_loss_epoch=0.700]#015Epoch 0:  10%|█         | 60/574 [00:07<01:01,  8.36it/s, loss=1.260, v_num=0, val_loss_epoch=0.700]#015Epoch 0:  10%|█         | 60/574 [00:07<01:01,  8.36it/s, loss=1.150, v_num=0, val_loss_epoch=0.700]#015Epoch 0:  16%|█▌        | 90/574 [00:10<00:57,  8.47it/s, loss=1.150, v_num=0, val_loss_epoch=0.700]#015Epoch 0:  16%|█▌        | 90/574 [00:10<00:57,  8.47it/s, loss=1.410, v_num=0, val_loss_epoch=0.700]#015Epoch 0:  21%|██        | 120/574 [00:14<00:53,  8.49it/s, loss=1.410, v_num=0, val_loss_epoc

#015                                                           #033[A#015Epoch 1:   0%|          | 0/574 [00:00<?, ?it/s, loss=1.510, v_num=0, val_loss_epoch=0.490, val_loss_step=0.525]          #015Epoch 2:   0%|          | 0/574 [00:00<?, ?it/s, loss=1.510, v_num=0, val_loss_epoch=0.490, val_loss_step=0.525]#015Epoch 2:   5%|▌         | 30/574 [00:03<01:07,  8.05it/s, loss=1.510, v_num=0, val_loss_epoch=0.490, val_loss_step=0.525]#015Epoch 2:   5%|▌         | 30/574 [00:03<01:07,  8.05it/s, loss=1.110, v_num=0, val_loss_epoch=0.490, val_loss_step=0.525]#015Epoch 2:  10%|█         | 60/574 [00:07<01:01,  8.35it/s, loss=1.110, v_num=0, val_loss_epoch=0.490, val_loss_step=0.525]#015Epoch 2:  10%|█         | 60/574 [00:07<01:01,  8.35it/s, loss=1.570, v_num=0, val_loss_epoch=0.490, val_loss_step=0.525]#015Epoch 2:  16%|█▌        | 90/574 [00:10<00:57,  8.38it/s, loss=1.570, v_num=0, val_loss_epoch=0.490, val_loss_step=0.525]#015Epoch 2:  16%|█▌        | 90/574 [00:10<00:57,  8.38it/s, lo

...

#015                                                           #033[A#015Epoch 2: 100%|██████████| 213/213 [00:41<00:00,  5.17it/s, loss=3.310, v_num=0, val_loss_epoch=0.189, val_loss_step=0.217]--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
#015Validation sanity check: 0it [00:00, ?it/s]#015Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]#015                                                              #015#015Training: 28it [00:00, ?it/s]#015

#015                                                           #033[A#015Epoch 1:   0%|          | 0/369 [00:00<?, ?it/s, loss=0.727, v_num=0, val_loss_epoch=0.281, val_loss_step=0.247]          #015Epoch 2:   0%|          | 0/369 [00:00<?, ?it/s, loss=0.727, v_num=0, val_loss_epoch=0.281, val_loss_step=0.247]#015Epoch 2:   8%|▊         | 30/369 [00:03<00:42,  7.94it/s, loss=0.727, v_num=0, val_loss_epoch=0.281, val_loss_step=0.247]#015Epoch 2:   8%|▊         | 30/369 [00:03<00:42,  7.94it/s, loss=1.550, v_num=0, val_loss_epoch=0.281, val_loss_step=0.247]#015Epoch 2:  16%|█▋        | 60/369 [00:07<00:37,  8.35it/s, loss=1.550, v_num=0, val_loss_epoch=0.281, val_loss_step=0.247]#015Epoch 2:  16%|█▋        | 60/369 [00:07<00:37,  8.35it/s, loss=1.520, v_num=0, val_loss_epoch=0.281, val_loss_step=0.247]#015Epoch 2:  24%|██▍       | 90/369 [00:10<00:33,  8.43it/s, loss=1.520, v_num=0, val_loss_epoch=0.281, val_loss_step=0.247]#015Epoch 2:  24%|██▍       | 90/369 [00:10<00:33,  8.43it/s, lo

#01525it [00:02, 13.07it/s]                        #033[A#01525it [00:02, 10.82it/s]

#015100%|██████████| 268/268 [00:32<00:00,  8.27it/s]#033[A#015269it [00:32,  8.31it/s]                         

#015100%|██████████| 268/268 [00:31<00:00,  8.06it/s]#033[A#015269it [00:31,  8.47it/s]                         

#015100%|██████████| 268/268 [00:31<00:00,  8.53it/s]#033[A#015269it [00:31,  8.48it/s]                         

#015100%|██████████| 268/268 [00:31<00:00,  8.56it/s]#033[A#015269it [00:31,  8.46it/s]                         

#015100%|██████████| 268/268 [00:31<00:00,  8.43it/s]#033[A#015269it [00:31,  8.47it/s]                         

#015 22%|██▏       | 60/268 [00:07<00:24,  8.63it/s]#033[A


#015100%|██████████| 268/268 [00:31<00:00,  8.39it/s]#033[A#015269it [00:31,  8.45it/s]                         

#015100%|██████████| 268/268 [00:31<00:00,  8.43it/s]#033[A#015269it [00:31,  8.50it/s]                         

#015Finding best initial lr: 100%|██████████| 100/100 [11:09<00:00,  6.69s/it]




2021-11-05 16:56:33 Uploading - Uploading generated training model
2021-11-05 16:58:34 Completed - Training job completed
Training seconds: 1741
Billable seconds: 1741
